<h3>0. Exploratory analysis</h3>

In [1]:
import json
import pandas as pd

df_training=pd.read_json('project_files/training.json')
df_devel=pd.read_json('project_files/devel.json')
df_docs=pd.read_json('project_files/documents.json')
df_testing=pd.read_json('project_files/testing.json')

print ("Done")

Done


In [ ]:
for item in df_training['text'][:1]:
    print(item)

In [ ]:
df_training.head()

In [ ]:
df_training.dtypes

In [ ]:
df_docs.head()

In [ ]:
df_docs.iloc[0]['text']

In [ ]:
df_testing.head()

In [ ]:
df_devel.head()

<h3>1. Question processing</h3>

In [ ]:
from collections import defaultdict
import nltk

def process_question(words):
    ## question type: classify the type of question
    # hand writter rule vs supervised ML - start with rules then we can move to supervised using datasets like.
    
    # default values
    question_type='UNK'
    answer_type='UNK'
    query=''
    
    # handwriten rules
    y_n_question=['is','can','could']
    wh_question=['what','who','where','when']
    
    for word in words:
        word=word.lower()
        if word in y_n_question:
            question_type='YN'
            break
        elif word in wh_question:
            question_type='WH'
            break
        else:
            question_type='UNK'
            
    
    ## focus: strings possible to be replaced in the answer
    # identify with the NER - use NLP taggin from CoreNLP
    
    ## answer type: kind of entity
    # from NER
    
    # just to start, the answer type is the same as question -- DELETE
    answer_type=question_type
    
    ## query: keywords to be used for the IR system to search in documents
    # do we need to do semantic parsing?
    # query should be built according to question type
    
    
    
    return query,answer_type

# process questions
queries=[]
for index, row in df_training.iterrows():
    question_training=nltk.word_tokenize(row['question'])
    query,answer_type=process_question(question_training)
    queries.append((query,answer_type))
    
    

In [ ]:
queries[1:4]

1. Find Keywords
2. Answer types - Using answer type taxonomy
3. Query formulation -> Keywords
4. Go to each document and check the frequency distribution of words and pick the document if one of the query words are present in document. Create a rank with that score
5. Find the paragraphs -> Discard irrelevant paragraphs. Use NE,Keywords, longest exact keywords. Put same weight for now and calculate the score of paragraphs. Rank each of the paragraphs in the document. We have to use the original answer and match the answer type
6. Find candidate answers -> Use supervised ML method
7. Merge candidate answers -> Use NER
8. Pick the best answer -> Logistic regression

<b>NER using NLTK OR Stanford CORENLP</b>

In [ ]:
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

Configuring Stanford CoreNLP . Link -> https://blog.manash.me/configuring-stanford-parser-and-stanford-ner-tagger-with-nltk-in-python-on-windows-f685483c374a

In [10]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tag import StanfordNERTagger

def get_Name_Entity_NLTK(data):
    results=[]
    for sentence in data:
        sentence = sentence.encode('utf8')
        ne_chunked_sents = ne_chunk(pos_tag(word_tokenize(unicode(sentence, errors='ignore'))))
        result = []
#         print (sentence)
        for tagged_tree in ne_chunked_sents:
#             print (tagged_tree)
            if hasattr(tagged_tree, 'label'):
                entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #
                entity_type = tagged_tree.label() # get NE category
                result.append((entity_name.decode('utf8'), entity_type))
        results.append(result)

    return results

def get_Name_Entity_Sentence(sentence):
    st = StanfordNERTagger('english.muc.7class.distsim.crf.ser.gz')
    sentence = sentence.encode('utf8')
    tokenized_text = nltk.word_tokenize(sentence)
    classified_text = st.tag(tokenized_text)
    result = {}
    
    for res in classified_text:
        if len(res) > 0:
            entity_name = res[0]
            entity_type = res[1]
            if len(entity_name) > 1 and len(entity_type) > 1:
                result[entity_name.strip()] = entity_type.strip()
    
    ne_chunked_sents = ne_chunk(pos_tag(word_tokenize(unicode(sentence, errors='ignore'))))
    for tagged_tree in ne_chunked_sents:
        if hasattr(tagged_tree, 'label'):
            entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #
            entity_type = tagged_tree.label() # get NE category
            result[entity_name.decode('utf8')] = entity_type
    
    return result

def get_Name_Entity_paragraph(paragraph):
    result = []
    tokenized_sentence = nltk.sent_tokenize(paragraph)
    for sentence in tokenized_sentence:
        result.append(get_Name_Entity_Sentence(sentence))
    
    return result

def get_Name_Entity_StanfordCoreNLP(data):
    st = StanfordNERTagger('english.muc.7class.distsim.crf.ser.gz')
    results=[]
    for sentence in data:
        sentence = sentence.encode('utf8')
        tokenized_text = nltk.word_tokenize(sentence)
        classified_text = st.tag(tokenized_text)
        result = []
        for res in classified_text:
            if len(res) > 0:
                entity_name = res[0]
                entity_type = res[1]
                if len(entity_name) > 1 and len(entity_type) > 1:
                    result.append([entity_name.strip(), entity_type.strip()])
        print (sentence, res)
        results.append(result)
        
    return results

def addNameEntity(df,feature,func):
    if 'NE'+"_"+feature in df:
        df = df.drop('NE'+"_"+feature, axis=1)
    df["NE"+"_"+feature] = func(df[feature])
    
    return df

Add named entity of questions to training data

In [6]:
df_training = addNameEntity(df_training,"question",get_Name_Entity_NLTK)

Add named entity of question answers to training dataframe. (Not needed for now)

In [ ]:
df_training = addNameEntity(df_training,"text",get_Name_Entity_StanfordCoreNLP)

<b> GET KEYWORDS</b>

In [16]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
POS = set(["NN","NNS","NNP","NNPS","CD","JJ","VB","VBD","VBG","VBN","VBP","VBZ"]) 
#returns lemmatized word

def lemmatize(word):
    word = unicode(word, errors='ignore')
    lemma = lemmatizer.lemmatize(word,wn.NOUN)
    if (lemma == word):
        lemma = lemmatizer.lemmatize(word,wn.VERB)
        
    return lemma

def get_keyword(data):
    result = []
    sentence = data.encode('utf8')
    tokenized_text = tokenizer.tokenize(sentence)
    tagged = nltk.pos_tag(tokenized_text)
    for text,pos in tagged:
        text = lemmatize(text.lower())
        if text not in stopwords.words('english'):
            if pos in POS:
                result.append(text)
                
    return result

def get_keyword_paragraph(data):
    results=[]
    tokenized_sentence = nltk.sent_tokenize(data)
    for sentence in tokenized_sentence:
        result = get_keyword(sentence)
        results.append(result)
        
    return results

def get_keyword_all(data):
    results=[]
    for sentence in data:
        result = get_keyword(sentence)
        results.append(result)
        
    return results

def add_keywords(df,feature):
    if 'keywords'+"_"+feature in df:
        df = df.drop('keywords'+"_"+feature, axis=1)
    df['keywords'+"_"+feature]=get_keyword_all(df[feature])
    return df

In [17]:
df_training = add_keywords(df_training,'question')
print ("Done")

Done


<b> GET POS</b>

In [18]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tag import StanfordNERTagger
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import StanfordPOSTagger

stanford_tagger = StanfordPOSTagger('english-bidirectional-distsim.tagger')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

def get_POS_paragraph(data):
    result = []
    tokenized_sentence = nltk.sent_tokenize(data)
    for sentence in tokenized_sentence:
        result.append(get_POS(sentence))
    return result
    
def get_POS(data):
    tokenized_text = tokenizer.tokenize(data)
    nltk_tag = nltk.pos_tag(tokenized_text)
    stan_tag = stanford_tagger.tag(tokenized_text)
    return stan_tag

<b>Exploring Question Types</b>

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

questionwords = set(["who", "what", "where", "when", "why", "how", "whose", "which", "whom","whats","what's","whos"])
for index,row in df_training.sort_values(by=['question']).iterrows():
    sentence = row["question"]
    found = False
    tokenized_text = tokenizer.tokenize(sentence)
    for text in tokenized_text:
        if text.lower() in questionwords:
            found = True
            break;
    if (not found):
        print (index,sentence,row["text"])

<b>Exploring Answer Paragraphs</b>

In [ ]:
doc = df_docs.iloc[0]['text']
paragraph = doc[22]
data = nltk.sent_tokenize(paragraph)
for sentence in data:
    result = get_Name_Entity_Sentence(sentence)
    print (sentence)
    print (result)
    print (get_keyword(sentence))

In [101]:
paragraph = doc[22]

print (get_keyword_paragraph(paragraph))
print (get_POS_paragraph(paragraph))
print (get_Name_Entity_paragraph(paragraph))

<b>Exploring Questions</b>

In [115]:
# Hardcoded word lists
questionwords = set(["who", "what", "where", "when", "why", "how", "whose", "which", "whom","whats","what's","whos"])
passiveQuestions = ["can", "could", "would", 
                    "was", "were","am","is", "are", "will","shall",
                    "did","do","does",
                    "had", "have","has",
                    "as","that","in",
                    "give an example","name"]

In [ ]:
for question in df_training['question'][0:5]:
    print (get_Name_Entity_Sentence(question))
    print (get_keyword(question))

In [ ]:
for index,row in df_training.iterrows():
    question = row["question"]
    answer = row["text"]
    print (question,answer.encode('utf-8'))
    print ("*****QUESTION*****")
    print (get_POS(question))
    print (get_keyword(question))
    print (get_Name_Entity_Sentence(question))
    print ("*****Answer*****")
    print (get_POS(answer))
    print (get_keyword(answer))
    print (get_Name_Entity_Sentence(answer))
    
    break

 2. Candidate Answering Generation

<h3>3. Candidate answering scoring</h3>

<h3>4. Answer and confidence</h3>